In [ ]:
# model load






In [ ]:
# data load

In [ ]:
# 해당 데이터에 대한 faster rcnn box visualization
# 기본 model 로드 코드
# self.model = fasterrcnn_resnet50_fpn(pretrained=True)
# self.model.roi_heads.nms_thresh=0.7



In [ ]:
# model에 들어가는 값이 model(img_feat, img_mask, qst)
# img_mask = True True .... True False False ... False가 들어가야 하고
# img_feat에 순서가 faster rcnn이 반환한 박스 순서랑 같음
# attention에 값과 faster rcnn에서 리턴된 box를 조인하여 그리기